In [1]:
import base64
import json

import pandas as pd
import requests
import tensorflow as tf

In [2]:
df = pd.read_csv("data/Reddit_Combi.csv")

df.head()


,text,label
0,Envy to other is swallowing me Im from develop...,1
1,Nothin outta the ordinary. Paradise. Job stres...,1
2,Almost 49 and the chasm of emptiness has never...,1
3,I’m happy again After my closest friend left m...,0
4,Is it possible to recover from such a traumati...,1


In [3]:
test_text = df["text"][0]

test_text


"Envy to other is swallowing me Im from developingcountry, Indonesia , and for now i temporary work overseas for 3 years contract, it's a hard labor job, and stressful. Next year my contract is finish. But, during my stay here, because of job, and my social life, my depression got worse, and i envy this developed country. Why this country is so good. I can afford anything i want here. Why can't we just have equality in currency exchange? I I need to work 15-20 years in big company in jakarta(our capital city) , just to get equal amount of saving money from what i got from 3 years working here. Yes, that's right, it's saving money, not spending money. And yes, im going to be a rich person in such young age If I think about it, this society is sick, the gap of un equality beetwen developing vs developed country, or the poor vs the rich is too big right now. Sorry if i look like an evil person , but because of this,I almost wish for war to happen, or this world to end, and got reseted. So

In [4]:
def prepare_json(text):
    feature_spec = {
        "text": tf.train.Feature(
            bytes_list=tf.train.BytesList(value=[bytes(text, "utf-8")])
        )
    }

    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()

    result = [{"examples": {"b64": base64.b64encode(example).decode()}}]

    return json.dumps({"signature_name": "serving_default", "instances": result})


In [5]:
def make_predictions(text):
    json_data = prepare_json(text)

    endpoint = "http://localhost:8080/v1/models/stress-detection-model:predict"
    response = requests.post(endpoint, data=json_data)

    prediction = response.json()["predictions"][0][0]
    return prediction


In [6]:
# Prediction
make_predictions(test_text)


0.952602267

In [7]:
# Actual label
df["label"][0]

1